In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import re
from tqdm import  tqdm
import os
import collections
import gc
from nltk.probability import ConditionalFreqDist, ConditionalProbDist, MLEProbDist, FreqDist
import math

In [2]:
file_path = "/kaggle/input/amharic-corpus-general/GPAC.txt"
n = 4

In [3]:
os.path.exists(file_path)
if os.path.exists(file_path):
    print(os.stat(file_path).st_size / (1024 * 1024))
else:
    print(os.path.exists(file_path))

1013.849326133728


# 1) N-gram language model

## 1.1 Create n-grams for n=1, 2, 3, 4. You can show sample prints.

In [27]:
# Downloading necessary NLTK data
nltk.download('punkt')

def process_chunk(chunk, n):
    tokens = word_tokenize(re.sub(r'\W+', ' ', chunk))
    return list(ngrams(tokens, n))

def read_and_process_in_chunks(file_path, n, chunk_size=100*1024*1024): #read about 100mb of data
    with open(file_path, 'r') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            return process_chunk(chunk, n)


all_ngrams = []
ngrams_chunk = read_and_process_in_chunks(file_path, n)
all_ngrams.extend(ngrams_chunk)
gc.collect()


[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


1785

In [28]:
print(len(all_ngrams))

19709367


In [29]:
print(all_ngrams[:10])

[('ምን', 'መሰላችሁ', 'አንባቢያን', 'ኢትዮጵያ'), ('መሰላችሁ', 'አንባቢያን', 'ኢትዮጵያ', 'በተደጋጋሚ'), ('አንባቢያን', 'ኢትዮጵያ', 'በተደጋጋሚ', 'ጥሪው'), ('ኢትዮጵያ', 'በተደጋጋሚ', 'ጥሪው', 'ደርሷት'), ('በተደጋጋሚ', 'ጥሪው', 'ደርሷት', 'ልትታደመው'), ('ጥሪው', 'ደርሷት', 'ልትታደመው', 'ያልቻለችው'), ('ደርሷት', 'ልትታደመው', 'ያልቻለችው', 'የአለም'), ('ልትታደመው', 'ያልቻለችው', 'የአለም', 'የእግር'), ('ያልቻለችው', 'የአለም', 'የእግር', 'ኳስ'), ('የአለም', 'የእግር', 'ኳስ', 'ዋ')]


## 1.2 Calculate probabilities of n-grams and find the top 10 most likely n-grams for all n.

In [31]:
# Create a conditional frequency distribution of Amharic n-grams
def create_freq_dist(ngrams):
    cfreq = ConditionalFreqDist((tuple(ngram[:-1]), ngram[-1]) for ngram in ngrams)
    return cfreq

# Create a conditional probability distribution using maximum likelihood estimation
def create_prob_dist(cfreq):
    cprob = ConditionalProbDist(cfreq, MLEProbDist)
    return cprob


probabilites = []

cfreq = create_freq_dist(all_ngrams)
cprob = create_prob_dist(cfreq)
probabilities = cprob
gc.collect()


0

In [32]:
# Get the top n-grams with the highest probabilities
def get_top_ngrams(cfreq, n, top_k):
    top_ngrams = []
    for context in cfreq.conditions():
        if len(context) == n - 1:
            freq_dist = cfreq[context]
            top_ngrams.extend([context,x,freq_dist.prob(x)] for x in freq_dist.samples())
    top_ngrams.sort(key = lambda x:x[2])
    return top_ngrams[len(top_ngrams)-10:]

In [33]:
top_ngrams = get_top_ngrams(probabilities, 4, 10)
for context, word, probability in top_ngrams:
    print(context,word,probability)

('እና', 'ህዝቦች', 'ወደ') አስከፊ 1.0
('ህዝቦች', 'ወደ', 'አስከፊ') ሁኔታ 1.0
('አስከፊ', 'ሁኔታ', 'እየወረዱ') ያሉባት 1.0
('ሁኔታ', 'እየወረዱ', 'ያሉባት') አገር 1.0
('እየወረዱ', 'ያሉባት', 'አገር') ሆናለች 1.0
('አገር', 'ሆናለች', 'ጆርጅ') ኦርዌል 1.0
('ሆናለች', 'ጆርጅ', 'ኦርዌል') በ1984 1.0
('ኦርዌል', 'በ1984', 'መፅፋቸው') ላይ 1.0
('በ1984', 'መፅፋቸው', 'ላይ') እንዳሉት 1.0
('መፅፋቸው', 'ላይ', 'እንዳሉት') ኢትዮጵያ 1.0


## 1.3 What is the probability of the sentence. "ኢትዮጵያ ታሪካዊ ሀገር ናት ". You can also try more sentences.

In [34]:
sentence = ("ኢትዮጵያ" ,"ታሪካዊ" ,"ሀገር", "ናት")
# Calculate the probability of a specific four-gram
def calculate_fourgram_probability(cprob, fourgram):
    context = tuple(fourgram[:-1])
    word = fourgram[-1]
    probability = cprob[context].prob(word)
    return probability

probability = calculate_fourgram_probability(probabilities, sentence)
probability

0

## 1.4 Generate random sentences using n-grams; explain what happens as n increases, based on your output.

In [39]:
# Generate a sentence using the n-gram model
def generate_sentence(cprob, max_length=20):
    sentence = []
    context = cprob.conditions()[0]
    while len(sentence) < max_length:
        word = cprob[context].generate()
        sentence.append(word)
        context = context[1:] + (word,)
        if word == '።':
            break
    return ' '.join(sentence)

In [40]:
generated_sentence = generate_sentence(probabilities)
print(generated_sentence)

ኢትዮጵያ በተደጋጋሚ ጥሪው ደርሷት ልትታደመው ያልቻለችው የአለም የእግር ኳስ ዋንጫን ሊያስተናግዱ የተወጠኑት ግዙፍ ስታዲየሞች ወደብና የምድር ውስጥ የባቡር ትራንስፖርት መስመር


# 2 Evaluate these Language Models Using Intrinsic Evaluation Method

In [37]:
def calculate_perplexity(cpdist, test_set):
    log_probabilities = [-math.log(cpdist[seq[:-1]].prob(seq[-1],)) for seq in test_set]
    perplexity = math.exp(sum(log_probabilities) / len(log_probabilities))
    return perplexity

test_set = all_ngrams[:30]

perplexity = calculate_perplexity(probabilities, test_set)
print("Perplexity:", perplexity)


Perplexity: 1.1200499091501965


# 3 Evaluate these Language Models Using Extrinsic Evaluation Method

In [38]:
def calculate_perplexity(prob_dist, test_data, chunk_size):
    total_log_probability = 0
    word_count = 0

    with open(test_data, 'r', encoding='utf-8') as file:
        chunk = file.read(chunk_size)
        tokens = nltk.word_tokenize(chunk)
        word_count += len(tokens)

        for token in tokens:
            prob = prob_dist[token].prob(token)
            if prob > 0:
                total_log_probability += math.log2(prob)

        chunk = file.read(chunk_size)

    avg_log_probability = total_log_probability / word_count
    perplexity = 2 ** (-avg_log_probability)
    return perplexity


chunk_size = 1024 * 10 * 1024

perplexity = calculate_perplexity(probabilities, file_path, chunk_size)#read another chunk from the file
print(f"Perplexity: {perplexity:.2f}")

Perplexity: 1.00
